# This notebook incorporate the Killercopscanada data + a data from news artcles found from google

In [574]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dateparser
import datefinder
import datetime
import os
import json
import re
import difflib
import string
from IPython.display import clear_output
from dateparser.search import search_dates
from dateparser_data.settings import default_parsers
parsers = [parser for parser in default_parsers if parser != 'relative-time']
# >>> parse('today', settings={'PARSERS': parsers})

can_province_names = {
      'Alberta':'AB',
      'British Columbia':'BC',
      'Manitoba':'MB',
      'New Brunswick':'NB',
      'Newfoundland and Labrador':'NL',
      'Newfoundland':'NL',
      'Nova Scotia':'NS',
      'Northwest Territories':'NT',
      'Nunavut':'NU',
      'Ontario':'ON',
      'Prince Edward Island':'PE',
      'Quebec':'QC',
      'Saskatchewan':'SK',
      'Yukon':'YT',
    }


can_province_keys = {
      'AB':'Alberta',
      'BC':'British Columbia',
      'MB':'Manitoba',
      'NB':'New Brunswick',
      'NL':'Newfoundland and Labrador',
      'NS':'Nova Scotia',
      'NT':'Northwest Territories',
      'NU':'Nunavut',
      'ON':'Ontario',
      'PE':'Prince Edward Island',
      'QC':'Quebec',
      'SK':'Saskatchewan',
      'YT':'Yukon',
    }


def Assign_Death_Code(df,Keys,Cols,Type,Recat=999,Validate = False,ValCols=None):
    for k in Keys:
        for c in Cols:
            if Validate == True:
                print(df.loc[((df['Death_Category']==Recat)&
                    (df[c].str.contains(k, flags=re.IGNORECASE, regex=True))),
                   ValCols].values)
            else:
                df.loc[((df['Death_Category']==Recat)&
                    (df[c].str.contains(k, flags=re.IGNORECASE, regex=True))),
                   'Death_Category']=Death_Catageory_Code[Type]
            
def FindDates(ss):
    flag = 'Parsed'
    
    DD = re.findall (r'\((.*?)\)',ss)
    for d in DD:
        if len(d)<=4:
            ss=ss.replace('('+d+')','')
    DD = re.findall(r'(?<=-)\w+',ss)
    for d in DD:
        ss = ss.replace('-'+d,'')
    s = ss.replace('(',' ').replace(')',' ').replace('.','').replace(',','').replace('$1500','')
#     print(s)
    Result=search_dates(s,languages=['en','fr'], settings={'PARSERS': parsers,
                                                           'PREFER_DAY_OF_MONTH': 'first',
                                                           'PREFER_DATES_FROM': 'past',
#                                                            'REQUIRE_PARTS': ['month', 'year']
                                                          })
    if Result is not None:
        if len(Result)>1:
#             print(d)
            print(ss)
            print(Result)
        date = Result[0][-1]
    else:
        date = None
        
        
#     Date=Result#[1]
    
# #     d = re.search(r'\((.*?)\)',s).group(1)
# #     print(d)
#     if '(' in s and ')' in s:
# #         print(re.findall (r'\((.*?)\)',s))
#         D = re.findall (r'\((.*?)\)',s)
#         d = ''
#         for DD in D:
# #             print(any(char.isdigit() for char in DD))
#             if ((len(DD)>6)and (any(char.isdigit() for char in DD))):
#                 d+= DD
#     else:
#         d=''
#     if len(d)>6:
#         date = dateparser.parse(d)
# #     else:
# #         date = None
#         if date is None:
#             try:
#                 sp = d.split(',')
#                 sp1 = sp[-1]
#                 sp0 = sp[0].split(',')[-1]
#                 d = sp0.split('-')[0].split('/')[0]+sp1#[-1]
# #                 d = sp[0].split('-')[-2].split('/')[0]+sp[-1]
#                 date = dateparser.parse(d)
#             except:
#                 dates = datefinder.find_dates(s)
#                 D = []
#                 for d in dates:
#                     D.append(d)
#                 if len(D) > 0:
#                     date = D[-1]
#                     flag = 'Found'
#                 else:
#                     date = None
#                     flag = 'Missing'
#                 pass
#     else:
#         dates = datefinder.find_dates(s)
#         D = []
#         for d in dates:
#             D.append(d)
#         print(s)
#         print(D,search_dates(s,languages=['en'], settings={'PARSERS': parsers,
#                                                            'PREFER_DAY_OF_MONTH': 'first',
#                                                            'PREFER_DATES_FROM': 'past',
#                                                            'REQUIRE_PARTS': ['month', 'year']}))
#         if len(D) > 0:
#             date = D[-1]
#             flag = 'Found'
#         else:
#             date = None
#             flag = 'Missing'
# #     print(date)
    return(date,flag)
            
Death_Catageory_Code = {
    'Shooting':1,
#     'Rubber Bullet':2,
    'Taser':3,
    'Other Weapon':4,
    'Excited Delirium':9,
    'Use of Force':10,
    'Accidental':15,
    'Overdose/Intoxication':25,
    'Vehicle':50,
    'In Custody':75,
    'Suicide':100,
    'Other/Unknown':999
}
Death_Category_SearchKeys = {
    'Shooting':['shoot','shot ','gunshot','rubber bullet','fired'],
#     'Rubber Bullet':[],
    'Taser':['tase','taze','stun gun'],
    'Other Weapon':['baton','club','pepper spray','pepper-spray'],
    'Excited Delirium':['excited delirium'],    
    'Use of Force':['Physical force','restrain','subdue','struggling','struggled','resisted','altercation',
                    'broken rib','collapsed lung',"beat and kick",'dragged','held him down'
#                     'handcuff','apprehended',
#                     'arrested','Anoxic brain injury','interact','trauma','bleeding','blood','strangling',
#                     'head injury','collapsed lung','pushed','fight','resisting','kicked','altercation',
#                     'respiratory failure','injuries'
                   ],
    'Accidental':['Fall','Fell','jump','hypothermia','drowning','suffocat'],
    'Overdose/Intoxication':['Methamphetamine','Cocaine','methadone','alcohol','pills','drugs',
                 'intoxication','overdose','toxicity','blocked with vomit'],
    'Vehicle':['crash','vehicle',"hit by officer's car",'collision'],
    'In Custody':['cell','custody','hospital','Died of an allergy','Found unresponsive','Segregation'],
    'Suicide':['suicide','suicidal','hang','hung','self-inflicted','Incisions of neck'],
    'Other/Unknown':['Unknown']
}

# Month_Abb{}

# Killer Cops Canada Data
* I've scraped this wordpress blog and saved the data (see other notebook) and saved the data as .json files

In [575]:
Length = []
Date = []
TitleDate = []
TitleDate_Flag = []
Title = []
Summary = []
ID = []
# Keys = []
Prov = []
URL = []
Categories = []
for i in range (1,31):
    filename = "killercopscanada/page"+str(i)+".json"

    with open(filename) as json_file:
        json_data = json.load(json_file)
        for post in json_data['posts']: 
#             if post['ID']== 1535:#_KCC
            Date.append(post['date'])
            Length.append(len(post['content']))
            Title.append(post['title'])
            Summary.append(post['content'])
            ID.append(str(post['ID'])+'_KCC')
#             Keys.append(list(post['tags'].keys()))
            Tag_Cat = list(post['categories'].keys())+list(post['tags'].keys())
            Categories.append(list(dict.fromkeys(Tag_Cat))) ## Remove duplicates
#             matches = datefinder.find_dates(post['title'].replace('Sept.','Sep.'))
            URL.append(post['URL'])
#             print(post['title'])
            date,flag = FindDates(post['title'])#\
#                     .replace('(Activists Killed by Cops Series)','Activists Killed by Cops Series').replace('(KRPF)','KPRF')\
#                     .replace('(Penticton, ','Penticton, (').replace('(Leduc, ','Leduc, (')\
#                     .replace('(Salluit, Nunavut, ','Salluit, Nunavut, (')\
# #                     .replace('(Black Lives Matter)','Black Lives Matter')\
#                     .replace('(Devon LaFleur, ','Devon LaFleur, (')\
#                     .replace('(Winnipeg, ','Winnipeg, ()')\
#                     .replace('(Brampton, Ontario, ','Brampton, Ontario, (')\
#                     .replace(', Arnprior)','), Arnprior')\
#                     .replace('(Toronto, ','Toronto, (')\
#                     .replace('(Toronto, ','Toronto, (')\
#                     .replace('(Sault Ste. Marie, ','Sault Ste. Marie, (')\
#                     .replace('(Morden, MB, ','Morden, MB, (')
#                     .replace('1-2','1').replace('6-7','6')
#                                  )
            TitleDate.append(date)
            TitleDate_Flag.append(flag)

KCC = pd.DataFrame(data = {'ID':ID,
                           'URL':URL,
                           'PostDate':Date,
                           'Title':Title,
                           'Categories':Categories,
#                            'Tags':Keys,
                           'PostLength':Length,
                           'Text':Summary,
                           'TitleDate':TitleDate,
                           'TitleDate_Flag':TitleDate_Flag})

KCC = KCC.set_index(pd.DatetimeIndex(KCC['PostDate']))
KCC['Match']=0
KCC.index=KCC.index.tz_localize(None)
# KCC.loc[((KCC.index-KCC['TitleDate']<datetime.timedelta(-1))&(KCC['TitleDate_Flag']=='Found')),'TitleDate']=np.nan

Man Shot by RCMP in Bonnington, BC, Dies Days Later (February 13, February 16, 2019)
[('February 13', datetime.datetime(2020, 2, 13, 0, 0)), ('February 16', datetime.datetime(2019, 2, 16, 0, 0)), ('2019', datetime.datetime(2019, 2, 1, 0, 0))]
Man Dies Following Police Interaction, IIO Discontinues Investigation (April 6/7, 2019)
[('April', datetime.datetime(2021, 4, 1, 0, 0)), ('6/7', datetime.datetime(2020, 6, 7, 0, 0)), ('2019', datetime.datetime(2019, 6, 1, 0, 0))]
First Police Killing of 2019: Edmonton Police Kill 26 Man (Jan. 2, 2019)
[('of 2019', datetime.datetime(2019, 5, 1, 0, 0)), ('Jan 2 2019', datetime.datetime(2019, 1, 2, 0, 0))]
Witness Identifies Victim of Hamilton Police Shooting as 19 (April 3, 2018)
[('19 April', datetime.datetime(2020, 4, 19, 0, 0)), ('3 2018', datetime.datetime(2018, 3, 1, 0, 0))]
Canada’s First Killing by Cop of 2017 as Montreal Police Kill Man on January 6
[('of 2017', datetime.datetime(2017, 5, 1, 0, 0)), ('on January 6', datetime.datetime(2017, 1

In [571]:
ss = 'July 7-8, 2020'
#     print(d)
#     if len(d)<=4:
#         ss=ss.replace('('+d+')',' ')

July 7, 2020


In [533]:
# print(KCC.loc[KCC['TitleDate_Flag']=='Missing'][-1:].values)
# print(KCC['Tags'])
KCC['prov']='Unspecified'

for i,row in KCC.iterrows():
    pc = 0
    for cat in row['Categories']:
        try:
            KCC.loc[KCC.index==i,'prov']=can_province_names[cat]
            pc += 1
        except:
            pass
    if pc > 1:
        KCC.loc[KCC.index==i,'prov']='Multiple'
# print(KCC['Categories'])
print(KCC.groupby('prov').count()['ID'])
T = KCC.loc[KCC['ID']=='1039_KCC']
print(T.index-T.TitleDate,T.TitleDate)
# print(KCC.loc[((KCC.index-KCC['TitleDate']<datetime.timedelta(-1))&(KCC['TitleDate_Flag']=='Found')),'TitleDate'])

In [534]:
#Filter out lists in KCC to get just posts about incidents
print(KCC.shape)
Excluders=['C.R.A.P', 
 'Police-Involved Deaths in Canada in 2017:',
 'People who Died Following a Police Intervention Since 1987',
 'Activists Killed by Cops',
 'A Victim of Power',
 'Third Consecutive Yearly Increase in Police-Involved Deaths in BC',
 'Deadly Force',
 'Delayed Information, Counterinformation',
 'IIO Report Documents Police Contempt for Oversight Agencies',
 'Still No Oversight, Still No Accountability', 
 'A Troubled Explanation']
for ex in Excluders:
    print(ex)
    if ex == 'C.R.A.P':
        KCC = KCC.loc[KCC['Text'].str.contains(ex)==False]
    else:
        KCC = KCC.loc[KCC['Title'].str.contains(ex)==False]
    print(KCC.shape)

(589, 11)
C.R.A.P
(562, 11)
Police-Involved Deaths in Canada in 2017:
(561, 11)
People who Died Following a Police Intervention Since 1987
(560, 11)
Activists Killed by Cops
(552, 11)
A Victim of Power
(552, 11)
Third Consecutive Yearly Increase in Police-Involved Deaths in BC
(551, 11)
Deadly Force
(550, 11)
Delayed Information, Counterinformation
(549, 11)
IIO Report Documents Police Contempt for Oversight Agencies
(548, 11)
Still No Oversight, Still No Accountability
(547, 11)
A Troubled Explanation
(546, 11)


# Missing Values
* This was my first attempt at filling the gaps, maunually creating a new spreadhseet & adding records
* Many of the records were pulled from killercopscanada blog.
* Some others were added from other sources
    * Where KCC is he linked source, I'll add the text post as the text summary and add it to PID


In [535]:
MD = pd.read_csv('MissingRecords_Skeeter_2021.csv',parse_dates=['Date'],index_col=['id_victim'])
MD = MD.fillna('')
# Find any records missing a summary and add it from the source
# I've already done for most records 
# print(MD[['summary','KCC_ID']])
for i, row in MD.loc[MD['data_source']!='killercopscanada'].iterrows():
    if row['summary'] == '':
        print(row['Link'])
        text = input('Enter Summary')
        MD.loc[MD.index==i,'summary']=text

# Any KCC posts, add the post text and URL
for i,row in MD.loc[MD['data_source']=='killercopscanada'].iterrows():
    if row ['summary'] == '':
        for l in row['Link'].split(' : '):
            if KCC.loc[KCC['URL']==l,'ID'].shape[0]>0:
    #             print()
                MD.loc[MD.index==i,'KCC_ID']=KCC.loc[KCC['URL']==l,'ID'].values[0]
                MD.loc[MD.index==i,'summary']=KCC.loc[KCC['URL']==l,'Text'].values[0]

# print(MD['summary'])
MD.to_csv('MissingRecords_Skeeter_2021.csv')
MD = MD.reset_index().set_index(pd.DatetimeIndex(MD.Date))

In [536]:
PID = pd.read_csv('CombinedSourceData.csv',parse_dates=['Date'])
PID = PID.reset_index(drop=True).set_index(pd.DatetimeIndex(PID.Date))
PID.shape

(1472, 35)

In [537]:
# Check if there are any "same" names within a 60 day window, allowing for typos.
# Check for missing names within a shorter widow, didnt find any tho
# Add any other records
# PID = PID.copy()
Window_named = 60
Window_Unnamed = 5
SimThresh=.75
# c = 0
# MD['Append'] = 1
MD['MD_match'] = 'None'
PID['MD_match'] = 'None'
# PID['CBC_id'] 
print(PID.count()['name'])
for i,row in MD.iterrows():
    if row['name']!='Unspecified':
        Match1=(difflib.get_close_matches(row['name'],PID['name'], cutoff=SimThresh))
        Subset = PID.loc[((np.abs(PID.index-i)<datetime.timedelta(Window_named))&
                               (PID['name'].isin(Match1))&
                               (PID['prov']==row['prov']))]
#         if row['name'] == 'Randy Trenholm':
#             print(row,Match1)
        if Subset.shape[0]>0:
            PID.loc[PID['id_victim']==Subset['id_victim'].values[0],'MD_match'] = row['id_victim']
            MD.loc[MD['id_victim']==row['id_victim'],'MD_match']=Subset['id_victim'].values[0]

PID = PID.append(MD)
# print(PID.loc[PID['MD_match']!='None'].shape)
# print(PID.loc[PID['MD_match']=='None'].shape)

# print(PID.columns)

PID = PID.set_index('id_victim')
# print(PID.loc[PID['name']=='Randy Trenholm'].index.values)
# print(PID.shape)
for i,row in PID.loc[((PID['MD_match']!='None'))].iterrows():
#     if i == '1251_CRAP' or i == 'MR_9':
#     print(i)
    Match = PID.loc[PID['MD_match']==i]
#     print(Match['prov'].values,row['prov'])
    if i.split('_')[-1]=='CRAP':
#         print(row)
#         print(Match.index)
        if Match['prov'].values[0] != row['prov']:
            print('?')
        else:
#             print(PID.shape)
#             print(Match,i)
            PID=PID.drop(i)
#     elif (row['MD_match'].split('_')[-1]!='CRAP') and (i.split('_')[0]=='MD') :
#         if 
    elif i.split('_')[0]!='MR':
#         print(row)
#         print(Match.index)
        PID.loc[(PID.index==i),'summary'] += PID.loc[(PID.index==i),'summary']
        PID.loc[(PID.index==i),'KCC_ID'] = PID.loc[(PID.index==i),'KCC_ID']
        PID=PID.drop(row['MD_match'])
#             print(PID.shape)
print(PID.columns)
print(PID.shape)
# print(PID.loc[PID['name']=='Randy Trenholm'])
PID['last_name']=PID['last_name'].fillna('Unspecified')

1472
Index(['Date', 'id_incident', 'day_week', 'prov', 'city_town',
       'address_intersection', 'postal_code', 'location_type', 'first_name',
       'middle_name', 'last_name', 'alias_nickname', 'age', 'gender', 'race',
       'ethnic_ancestry', 'immigrant_refugee_naturalized', 'armed_type',
       'cause_death', 'taser_deployed', 'injured_officer', 'excited_delirium',
       'mentral_distress_disorder', 'substance_abuse', 'summary', 'department',
       'charge_type', 'charges', 'officers involved', 'name', 'data_source',
       'Inquest date', 'GS_match', 'laCRAP_match', 'MD_match',
       'DEATH CATEGORY', 'Link', 'KCC_ID'],
      dtype='object')
(1650, 38)


In [538]:
# print(PID.loc[PID.index])
PID.loc[PID['KCC_ID']=='None','KCC_ID']=np.nan
PID.loc[PID['KCC_ID']=='','KCC_ID']=np.nan
print(PID.groupby('KCC_ID').count()['prov'].sort_values())
# print(KCC.loc[KCC['ID']=='993_KCC'])

PID = PID.reset_index(drop=False).set_index(pd.DatetimeIndex(PID.Date))
PID['KCC_ID']=PID['KCC_ID'].fillna('')
Exclude = PID.loc[((PID['KCC_ID']!='')),
              'KCC_ID'].values
# print(Exclude)
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
STOPWORDS.add('man')

for i,row in KCC.iterrows():
    T=row['Title'].lower()\
    .replace('[{0}]*'.format(string.punctuation), '')\
    .strip()\
    .split()
    List =[]
    for word in T:
        if word not in STOPWORDS:
            List.append(word)
#         List.append
    S = set(List)
#     print(S)
    KCC.loc[KCC.index==i,'JustWords_Title']=[S]

KCC_ID
1009_KCC     1
1739_KCC     1
1732_KCC     1
1729_KCC     1
1721_KCC     1
            ..
1279_KCC     2
1217_KCC     2
1155_KCC     2
997_KCC      2
993_KCC     33
Name: prov, Length: 140, dtype: int64


In [539]:

PID['middle_name']=PID['middle_name'].fillna('')
KCC['id_victim']=''
KCC['Match']=0.0
KCC['name']=''
Dated = KCC.loc[((KCC['TitleDate'].isnull()==False)&(KCC['ID'].isin(Exclude)==False))].copy()
NotDated = KCC.loc[((KCC['TitleDate'].isnull()==True)&(KCC['ID'].isin(Exclude)==False))].copy()
# print(KCC.shape)
print(Dated.shape)
print(NotDated.shape)
Named = PID.copy()#loc[PID.name!='Unspecified']

# for 
Window_d = 4
Thresh = .75
Dated = Dated.reset_index(drop=True).set_index(pd.DatetimeIndex(Dated.TitleDate))
# print(Dated.columns)
for i,row in Named.iterrows():#.loc[Named['id_victim']=='MR_205']
    Match = Dated.loc[np.abs(Dated.index-i)<=datetime.timedelta(Window_d)].copy()
#     print(Match)
    if Match.shape[0]>0 and row['name']!='Unspecified':
        
        LN=row['last_name'].lower()
        Match['match'] = Match.JustWords_Title.apply(lambda words: difflib.get_close_matches(LN,words, cutoff=Thresh))
        
        if Match.shape[0]>0:#.loc[Match['match'].str.len()!=0]
            for j,row2 in Match.loc[Match['match'].str.len()!=0].iterrows():
                Named.loc[Named.id_victim==row['id_victim'],'KCC_ID'] += ' ' + row2['ID']
                Named.loc[Named.id_victim==row['id_victim'],'summary'] += '       ' + row2['Text']
                Dated.loc[Dated['ID']==row2['ID'],'Match']+=1
                Dated.loc[Dated['ID']==row2['ID'],'id_victim']+=';'+row['id_victim']
                Dated.loc[Dated['ID']==row2['ID'],'name']+=';'+row['name']
#         else:
            for j,row2 in Match.loc[Match['match'].str.len()==0].iterrows():
                if can_province_keys[row['prov']] in row2['Categories']:
                    Named.loc[Named.id_victim==row['id_victim'],'KCC_ID'] += ' ***' + row2['ID']
                    Named.loc[Named.id_victim==row['id_victim'],'summary'] += '       ***' + row2['Text']
                    Dated.loc[Dated['ID']==row2['ID'],'Match']+=.0001
                    Dated.loc[Dated['ID']==row2['ID'],'id_victim']+=';**'+row['id_victim']
                    Dated.loc[Dated['ID']==row2['ID'],'name']+=';**'+row['name']

    elif Match.shape[0]>0:
        for j,row2 in Match.iterrows():
            if can_province_keys[row['prov']] in row2['Categories']:
                Named.loc[Named.id_victim==row['id_victim'],'KCC_ID'] += ' **' + row2['ID']
                Named.loc[Named.id_victim==row['id_victim'],'summary'] += '       **' + row2['Text']
                Dated.loc[Dated['ID']==row2['ID'],'Match']+=.01
                Dated.loc[Dated['ID']==row2['ID'],'id_victim']+=';**'+row['id_victim']
                Dated.loc[Dated['ID']==row2['ID'],'name']+=';**'+row['name']
    elif row['name']!='Unspecified': 
#         print(row['last_name'])
        LN=row['last_name'].lower()
        NotDated['match'] = NotDated.JustWords_Title.apply(lambda words: difflib.get_close_matches(LN,words, cutoff=Thresh))
#         print(NotDated['match'].sum())  
        for j,row2 in NotDated.loc[NotDated['match'].str.len()!=0].iterrows():
            if row['prov'] in row2['Categories']:
                print(row2['Title'])
                Named.loc[Named.id_victim==row['id_victim'],'KCC_ID'] += ' *' + row2['ID']
                Named.loc[Named.id_victim==row['id_victim'],'summary'] += '       *' + row2['Text']
                Dated.loc[Dated['ID']==row2['ID'],'Match']+=.1
                Dated.loc[Dated['ID']==row2['ID'],'id_victim']+=';**'+row['id_victim']
                Dated.loc[Dated['ID']==row2['ID'],'name']+=';**'+row['name']
print('Not Matched: ',Dated.loc[Dated['Match']==0].shape[0])
print('Matched: ',Dated.loc[Dated['Match']>0].shape[0])

(229, 14)
(179, 14)
“Comply or Die” Policing: Unanswered Questions in the Police Killing of Myles Gray
Mental Health, Police Escalation, and the Killing of Rhett Mutch
Few Details as RCMP Investigated in Death of Nicolas Jeppesen in Terrace, BC
Not Matched:  28
Matched:  201


In [542]:
Dated.loc[Dated['Match']==0,['TitleDate','Title']].values
# Manual_Match=[['1646_KCC','MR_189'],['1123_KCC','0178-V1'],['658_KCC','0131-V1'],['530_KCC','0064-V1']]
# for m in Manual_Match:
#     Named.loc[Named.id_victim==m[1],'KCC_ID'] += ' ' + m[0]
#     Named.loc[Named.id_victim==m[1],'summary'] += '       ' + Dated.loc[Dated['ID']==m[0],'Text']
#     Dated.loc[Dated['ID']==m[0],'Match']+=.01
#     Dated.loc[Dated['ID']==m[0],'id_victim']+=';'+m[1]#row['id_victim']
#     Dated.loc[Dated['ID']==m[0],'name']+=';'+Named.loc[Named.id_victim==m[1],'name']
    
# for i,row in Dated.loc[Dated['Match']==0].iterrows():
#     print(row['ID'],row['Title'],row['TitleDate'],row['URL'])

array([[Timestamp('2021-05-30 00:00:00'),
        'Seventy-Eight-Year-Old Man Dies During Police Intervention in Saint-Jérôme, Québec (Mar. 30, 2021)'],
       [Timestamp('2020-12-02 00:00:00'),
        'Father of 1-Year-Old Killed in OPP Shooting Dies in Hospital (Dec. 2, 2020)'],
       [Timestamp('2019-11-01 00:00:00'),
        'Clive Mensah Identified as Man Killed by Peel Police in November 2019'],
       [Timestamp('2020-07-01 00:00:00'),
        'Hamilton Police Shoot and Kill 42-Year-Old Man (July 7-8, 2020)'],
       [Timestamp('2017-05-01 00:00:00'),
        'Killer Cop Justin Holz Gets 30 Months for Killing Cody Severight in 2017'],
       [Timestamp('2019-09-22 00:00:00'),
        'Kingston Police Shoot and Kill Evan Freeman (22) (Sept. 12, 2019)'],
       [Timestamp('2020-02-13 00:00:00'),
        'Man Shot by RCMP in Bonnington, BC, Dies Days Later (February 13, February 16, 2019)'],
       [Timestamp('2019-05-28 00:00:00'),
        'Fifty-Year-Old Man Dies During Arrest 

In [364]:
print(PID.loc[PID.prov=='NL','department'])

Date
2015-04-05    Royal Newfoundland Constabulary
2000-08-26                               RCMP
2000-10-16    Royal Newfoundland Constabulary
2018-11-27    Royal Newfoundland Constabulary
1996-03-02                            Unknown
2018-04-13                            Unknown
Name: department, dtype: object


In [ ]:
# 

# print(KCC['JustWords_Title'][0])

# KCC['JustWords_Text'] = [set(words) for words in
#     KCC['Text']
#     .str.lower()
#     .str.replace('[{0}]*'.format(string.punctuation), '')
#     .str.strip()
#     .str.split()
# ]
print(KCC['JustWords_Title'].values)

# Find Matching records in KCC
* This is the next step, match up existing recrods.  Append missing.

In [ ]:

PID['middle_name']=PID['middle_name'].fillna('Unspecified')
KCC['Match']=0
KCC['id_victim']=''
KCC['name']=''
KCC['Exclude'] = False
for ex in Exclude:
    KCC.loc[KCC.ID==ex,'Exclude']=True
    KCC.loc[KCC.ID==ex,'Match']+=1
    KCC.loc[KCC.ID==ex,'id_victim']=PID.loc[PID.KCC_ID==ex,'id_victim'].values[0]
    KCC.loc[KCC.ID==ex,'name']=PID.loc[PID.KCC_ID==ex,'name'].values[0]
# KCC_BU = KCC.copy()
KCC_EX = KCC.loc[KCC['Exclude']==True]
KCC = KCC.loc[KCC['Exclude']==False]

print(KCC.shape)
print(KCC_EX.shape)
LThresh=.85
FMThresh=.7
# SimThresh=.75
# PID['KCC']
for i,row in PID.loc[((PID.name!='Unspecified'))].iterrows():
#     print(row['id_victim'])
    LN=row['last_name'].lower()#.copy()
    FN=row['first_name'].lower()#.copy()
    MN=row['middle_name'].lower()#.copy()
    if len(MN)<=2:
        MN = 'Unspecified'
#     print(len(MN),'   ',MN)
    KCC['matchL'] = KCC.JustWords_Title.apply(lambda words: difflib.get_close_matches(LN,words, cutoff=LThresh))
    KCC['matchF'] = KCC.JustWords_Title.apply(lambda words: difflib.get_close_matches(FN,words, cutoff=FMThresh))
    KCC['matchM'] = KCC.JustWords_Title.apply(lambda words: difflib.get_close_matches(MN,words, cutoff=FMThresh))
    if KCC.loc[(((KCC['matchL'].str.len()!=0)&(KCC['matchF'].str.len()!=0))|
               ((KCC['matchL'].str.len()!=0)&(KCC['matchM'].str.len()!=0)))].shape[0]>0:
#         print(row['name'])
#         print(KCC.loc[((KCC['matchL'].str.len()!=0)&(KCC['matchF'].str.len()!=0)),'Title'].values)
        for j,row2 in KCC.loc[(((KCC['matchL'].str.len()!=0)&(KCC['matchF'].str.len()!=0))|
               ((KCC['matchL'].str.len()!=0)&(KCC['matchM'].str.len()!=0)))].iterrows():
            PID.loc[PID.id_victim==row['id_victim'],'KCC_ID'] += ' ' + row2['ID']
            PID.loc[PID.id_victim==row['id_victim'],'summary'] += '       ' + row2['Text']
        KCC.loc[(((KCC['matchL'].str.len()!=0)&(KCC['matchF'].str.len()!=0))|
               ((KCC['matchL'].str.len()!=0)&(KCC['matchM'].str.len()!=0))),'Match']+=1
        KCC.loc[(((KCC['matchL'].str.len()!=0)&(KCC['matchF'].str.len()!=0))|
               ((KCC['matchL'].str.len()!=0)&(KCC['matchM'].str.len()!=0))),'id_victim']+=';'+row['id_victim']
        KCC.loc[(((KCC['matchL'].str.len()!=0)&(KCC['matchF'].str.len()!=0))|
               ((KCC['matchL'].str.len()!=0)&(KCC['matchM'].str.len()!=0))),'name']+=';'+row['name']
        

KCC = KCC.append(KCC_EX)
print(KCC.loc[KCC['Match']==0].shape)
print(KCC.loc[KCC['Match']>0].shape)

In [ ]:
# print(KCC.loc[KCC['Match']==0,['Title','TitleDate']].count())
print(KCC.loc[KCC['Match']>=2,['Title','name','prov']][0:20].values)
# print(KCC.loc)
print(PID.loc[PID['last_name']=='Campbell',['first_name','middle_name','KCC_ID','prov']])

# print(KCC.loc[KCC['Title'].str.contains('Killing of Dudley George')==True,['Title','Match']].values)

# Window_d=3

print(KCC.JustWords_Title.apply(lambda words: difflib.get_close_matches('Jason',words, cutoff=.7)).sum())
# KCC['matchF'] = KCC.JustWords_Title.apply(lambda words: difflib.get_close_matches('Dudley',words, cutoff=FMThresh))
# print(KCC.loc[(((KCC['matchL'].str.len()!=0)&(KCC['matchF'].str.len()!=0))|
#                ((KCC['matchL'].str.len()!=0)&(KCC['matchM'].str.len()!=0)))])
# KCC.loc[((KCC['matchL'].str.len()!=0)&(KCC['matchF'].str.len()!=0))]
# print(PID.loc[PID['KCC_ID'].str.contains('415_KCC')])
# print(KCC.loc[KCC['ID']=='415_KCC'])
# for i,row in KCC.loc[((KCC['Match']==0)&(KCC['TitleDate'].isnull()==True))].iterrows():
#     print(row['TitleDate'],row['prov'])
#     Match = PID.loc[((np.abs(PID.index-row['TitleDate'])<=datetime.timedelta(Window_d))&(PID['prov'] == row['prov']))]
#     print(Match.shape)

In [ ]:
# KCC['JustWords_Title'] = [set(words) for words in
#     KCC['Title']
#     .str.lower()
#     .str.replace('[{0}]*'.format(string.punctuation), '')
#     .str.strip()
#     .str.split()
# ]

# KCC['JustWords_Text'] = [set(words) for words in
#     KCC['Text']
#     .str.lower()
#     .str.replace('[{0}]*'.format(string.punctuation), '')
#     .str.strip()
#     .str.split()
# ]
# PID['middle_name']=PID['middle_name'].fillna('')
# for i,row in PID.loc[((PID.index.year>=2014)&(PID.KCC_ID!='Unspecified')&(PID.name!='Unspecified'))].iterrows():
# #     print(i)
#     target_words=[row['last_name']]
#     target_words = [word.lower() for word in target_words]
#     KCC['matchL'] = KCC.JustWords_Title.apply(lambda words: all(target_word in words 
#                                            for target_word in target_words))
#     if KCC['matchL'].sum()>0:
#         Set = KCC.loc[KCC['matchL']==True].copy()
#         target_words=[row['first_name'],row['middle_name']]
# #         print(target_words)
#         target_words = [word.lower() for word in target_words]
#         Set['matchFM'] = Set.JustWords_Title.apply(lambda words: any (target_word in words 
#                                                for target_word in target_words))
# #         Set['match'] = Set['matchL']+Set['matchFM']
# #         print(Set['match'].sum())
#         if Set['matchFM'].sum()>0:
#             print(Set['Title'],row['name'])

In [ ]:

# KCC['id_victim'] = ''
# KCC['Title_Match'] = 0
# KCC['Text_Match'] = 0
# KCC['Names']=''
# PID['Matches'] = 0# 
# # print(PID['Matches'])

# Title_Match = {}
# Text_Match = {}
# # print(PID['Matches'])
# R = 1
# PID['KCC_ID']=PID['KCC_ID'].fillna('')
# for i, row in PID.iterrows():
#     Title_Match[i]=[]
#     if row['KCC_ID']!='Unspecified':
#         KCC.loc[KCC['ID'] == row['KCC_ID'],'Match'] += 1
#     if 'Unspecified' not in row['name']:
#         target_words = [row['first_name'], row['last_name']]
# #         print()
# #         print(i,row['name'])
# #         print(target_words)
#         target_words = [word.lower() for word in target_words]
# #         if i=='264_CRAP':
# #             print(target_words)
#         KCC['match'] = KCC.JustWords_Title.apply(lambda words: all(target_word in words 
#                                                for target_word in target_words))
        
#         KCC.loc[KCC['match']==True,'Title_Match']+=1
# #         KCC_ID = row['KCC_ID']
#         KCC.loc[KCC['match']==True,'id_victim'] += i+' '
#         KCC.loc[KCC['match']==True,'Names']+=row['name']+' ; '
#         if KCC['match'].sum()>0:
#             for i2,row2 in KCC.loc[KCC['match']].iterrows():
#                 if row2['ID'] not in row['KCC_ID']:
#                     PID.loc[PID.index==i,'KCC_ID']+= ' '+row2['ID']
#                     PID.loc[PID.index==i,'summary']+= '    '+row2['Title']+'    '+row2['Text']
# #                     print(row['KCC_ID'],row2['ID'])
#         else:
#             target_words = [row['last_name']]
# #             print(target_words)
#             target_words = [word.lower() for word in target_words]
#             KCC['match'] = KCC.JustWords_Title.apply(lambda words: all(target_word in words 
#                                                    for target_word in target_words))
#             KCC.loc[((KCC['match']==True)&(KCC['Title_Match']==0)),'Text_Match']+=1
# #             KCC_ID = row['KCC_ID']
#             KCC.loc[((KCC['match']==True)&(KCC['Title_Match']==0)),'id_victim'] += i+' '
#             KCC.loc[((KCC['match']==True)&(KCC['Title_Match']==0)),'Names']+=row['name']+' ; '
#             if KCC['match'].sum()>0:
#                 for i2,row2 in KCC.loc[((KCC['match']==True)&(KCC['Title_Match']==0))].iterrows():
#                     if row2['ID'] not in row['KCC_ID']:
#                         PID.loc[PID.index==i,'KCC_ID']+= ' '+row2['ID']
#                         PID.loc[PID.index==i,'summary']+= '    '+row2['Title']+'    '+row2['Text']


In [ ]:
# print(PID.loc[PID.index=='83_GS',['first_name','middle_name','last_name']])
# print(PID.loc[PID['KCC_ID']== ''].shape)
KCC['Matches'] = KCC[['Title_Match','Text_Match']].sum(axis=1)

# print(KCC['Matches'].sort_values())
print(KCC.loc[KCC['Matches']==0].shape)
print(KCC.loc[KCC['Matches']>=1].shape)
# print(KCC.loc[KCC['Text'].str.contains('C.R.A.P')].shape)
# 

Exclude = PID.loc[((PID['KCC_ID']!='')),
              'KCC_ID'].values
print(Exclude.shape)
EX = []
for V in Exclude:
    for v in V.split(' '):
        if v !='' and v not in EX:
            EX.append(v)
    
print(len(EX))

Dated = KCC.loc[((KCC['TitleDate'].isnull()==False)&(KCC['ID'].isin(EX)==False))]
NotDated = KCC.loc[((KCC['TitleDate'].isnull()==True)&(KCC['ID'].isin(EX)==False))]

print(Dated.loc[Dated['Match']==0,'Title'].shape)
print(NotDated.loc[NotDated['Match']==0,'Title'].shape)
print(NotDated.loc[NotDated['Match']==0,'Title'].values)

print(KCC.loc[KCC['Title']=='A Troubled Explanation: Using Victims’ Histories of Mental Illness to Excuse Killer Cops','Text'].values)
# Excluders.append('A Troubled Explanation: Using Victims’ Histories of Mental Illness to Excuse Killer Cops')
# print(Excluders)



In [ ]:
print(KCC.loc[KCC['Title']=='A Troubled Explanation: Using Victims’ Histories of Mental Illness to Excuse Killer Cops','Text'].values)
# Excluders.append('A Troubled Explanation: Using Victims’ Histories of Mental Illness to Excuse Killer Cops')
# print(Excluders)

In [ ]:

Window_d=2
c = 0
# # print(KCC.loc[KCC['Match']==0,'ID'].count())
for i,row in Dated.iterrows():
    Match = PID.loc[((np.abs(PID.index-row['TitleDate'])<=datetime.timedelta(Window_d))&(PID['prov'] == row['prov']))]
    if Match['id_victim'].count()>=1:
        c +=1
        for v in Match['id_victim']:
#     #         print('/')
            print('Post')
            print(row[['ID','Matches']])
            print(row['Title'])
            print(row['Text'])
            print()
            print('Record')
#             print(Match['id_victim'].count(),Match['Matches'].sum())
            print(PID.loc[PID['id_victim']==v,'summary'].values)
            print(PID.loc[PID['id_victim']==v,['id_victim','name','age','department','city_town','gender','KCC_ID']])
#             print(Match['KCC_ID'])
        
        
            Match = input('Match: y/n')
            if Match == 'y':
                PID.loc[PID['id_victim']==v,'summary'] += '      '+ row['Text']
                PID.loc[PID['id_victim']==v,'KCC_ID'] += row['ID']+' '
                KCC.loc[KCC['ID']==row['ID'],'Matches']+=1
                KCC.loc[KCC['ID']==row['ID'],'id_victim']+=v+' '
#             print()
            clear_output()
print(c)

In [ ]:
# row = (PID.loc[PID.id_victim=='264_CRAP',['first_name','last_name']])

# target_words = [row['first_name'][0].lower(), row['last_name'][0].lower()]
# print(target_words)
# print(KCC.loc[KCC.ID=='1735_KCC','JustWords_Title'].values)
# # print(KCC.loc[KCC['ID']=='993_KCC','Tags'].values)
# KCC.JustWords_Title.apply(lambda words: all(target_word in words 
#                                                for target_word in target_words)).sum()

In [ ]:

print(PID.columns)

PID['Check']=True
PID.loc[(PID['postal_code']!='Unspecified'),'Check']=False
Summary = PID[['summary','data_source']]
Summary['extra_source'] = ''
Summary.to_csv('Compiled/Summary.csv')
Location = PID[['prov', 'city_town','address_intersection', 'postal_code','Check']]
Location.to_csv('Compiled/Location.csv')
Victim = PID[['name','first_name', 'middle_name', 'last_name', 'alias_nickname', 'age',
       'gender', 'race', 'ethnic_ancestry']]
Victim.to_csv('Compiled/Victim.csv')
Incident = PID[['Date','armed_type', 'cause_death','DEATH CATEGORY', 'taser_deployed', 'injured_officer',
       'excited_delirium', 'mentral_distress_disorder', 'substance_abuse','department',
        'charge_type', 'charges', 'officers involved','data_source', 'Inquest date']]

Incident.to_csv('Compiled/Incident.csv')
# print(Summary)

In [ ]:
print(PID.loc[PID['KCC_ID']=='Unspecified','prov'].count())
print(PID.loc[PID['KCC_ID']!='Unspecified','prov'].count())


In [ ]:
PID['Check']=True
PID.loc[(PID['postal_code']!='Unspecified'),'Check']=False

PID.index=PID.index.rename('Date')
PID.to_csv('Combined_Data.csv')
print(PID.columns)